# IVR8 Advanced Example
This example aims to highlight the follow:
 * demonstrate how to use compartment constraints on a particular model
 * Use a subset of the publist stops and configure a single vehicle
 * Use a simple **one-rack** comparment configuration
 * Add allowable-compartment assignments (i.e. which jobs may be assigned to which compartments)

## Requirements
This example assumes you've configured an api-key with the required services enabled (see the [portal](portal.icepack.ai) for configuration details) and that you're familiar with loading and working with protobuf models.

## Data
We're going to load sample data which has order sizes and durations in a tabular format. The first 9 items will be sufficient for this exercise (item 1 being the depot, and 8 for orders)


In [ ]:
import pandas
df = pandas.read_csv('../../sample_data/publist_orders.csv').head(9)
print(df.head())

## Model helper
As with the previous example, we'll be using the model helper and creating a top rack / lower rack style of compartments.

We're going to start by creating an ivr8 model and populating it with some basic information.

In [ ]:
exec(open('apiHelper.py').read()) # import some api-helper classes which we've written for you.
exec(open('ivr8-model-helper.py').read()) # import some modelling helpers

api = apiHelper(modelType="ivr8-yni1c9k2swof") # set the model type to the ivr8 model

sr = ivr8_yni1c9k2swof_pb2.SolveRequest()
sr.solveType = 0 # optimise solve request

m = sr.model
m.dimensions.CopyFrom(make_distance_time_cap_dims())
m.locations.extend(make_locations(df)) 
m.locations.append(m.locations[0]) # just copy the Guiness storehouse location
m.locations[len(m.locations) - 1].id = 'vehicle-site' #rename it and we're good to go (location id's need to be unique in a model)
m.jobs.extend(make_job_time_cap(df, [0] * (df.shape[0]-1), range(1, df.shape[0])))
m.vehicleCostClasses.append(make_vcc_simple('vcc1', 1000, 1.001e-2, 1.001e-2, 1.001e-2, 1, 3))
m.vehicleClasses.append(make_vc_simple('vc1', 1, 1, 1, 1))
m.vehicles.append(make_vehicle_cap('vehicle_1', 'vc1', 'vcc1', #create just one vehicle
                                    2000, # the vehicle capacity
                                    df['id'][0], # the start location
                                    df['id'][0], # the end location
                                    7*60,        # the start time (7AM)
                                    18*60))      # the end time  (6PM)



### Compartment definition
A compartment is a container in which tasks can be placed. Several compartments can be assigned to a vehicle, comprising what we call a `compartment-set`. We can place constraints on individual compartments from an assignmnet perspective (i.e. some tasks may only be assigned to certain compartments) as well as individual constraints on the capacity of a compartment (with respect to some defined dimensions). 

Lets make a simple example where we have a vehicle who's compartments are layed out as follows:


| Rack         |       |       |       |       |
| -------------|-------|-------|-------|-------|
| Lower Rack   | c1:500|c2:500 |c3:500 |c8:500 |

So we have 4 compartments in a rack (c1-c4) each with a capacity of 500 units.



In [ ]:
del m.compartments[:] # in case you run this twice
for i in range(0,4):
    comp = ivr8_yni1c9k2swof_pb2.Compartment()
    comp.id = "c" + str(i+1)
    compcap = ivr8_yni1c9k2swof_pb2.Compartment.Capacity()
    compcap.dimensionId = 'capacity'
    compcap.capacity = 400 
    comp.capacities.append(compcap)
    m.compartments.append(comp)
print(m.compartments)

### Compartment - task constraints
Now we can go back through the tasks and allocate them to allowable compartments. This is normal in fuel delivery systems where you have diesel/petrol constraints. We're just going to decide on which jobs may go in which compartments based on the index (because this example is fictional), and lets see if that's feasible. Obviously, you'll create it using proper logic based on the business rules.

In [ ]:
for i, j in enumerate(m.jobs):
    relation = ivr8_yni1c9k2swof_pb2.Job.CompartmentRelation()
    relation.type = 0 # 0 for inclusions, 1 for exclusions
    if (i + 1) % 2 == 0:
        relation.compartmentIds.append('c1')
        relation.compartmentIds.append('c3')
    else:
        relation.compartmentIds.append('c2')
        relation.compartmentIds.append('c4')
    j.compartmentRelations.CopyFrom(relation)

print(m.jobs[0]) # so here we can see we're allowed c2 and c4 as compartment assignments for this task

In [ ]:
del m.compartmentSets[:] # in case you run this twice
cset = ivr8_yni1c9k2swof_pb2.CompartmentSet()
cset.id = 'tanker'
for i in range(0,4):
    cset.compartmentIds.append('c' + str(i + 1)) #indicates that this compartment set has c1:c8 available.
m.compartmentSets.append(cset) # add it to the model

print('Compartment set:')
print(m.compartmentSets)
# Here we assign the compartment set to the vehicle class (which would then apply to all vehicles in that class) 
m.vehicleClasses[0].compartmentSetId = 'tanker'

## Run the model

Now we can follow the normal process of submitting the model to the api and running it!


In [ ]:
exec(open('apiHelper.py').read()) # import some api-helper classes which we've written for you.
reqId = api.Post(sr)
sol = api.Get(reqId)
t = tabulate(sr, sol)


### Reading the response
We can see in the tabulated response that there is now a `compartmentId` field which gives the allocation of tasks to compartments in the solution. This view of the response is useful, but we can check another view we've coded up to understand now the vehicle behaves over the course of the route.

Lets have a look at the `compartmentSummary` table:

In [ ]:
csummary = t['compartmentSummary']
csummary[list(csummary.keys())[0]] # just grab the compartment summary for the first vehicle.


The compartment summary is nice - but it doesn't tell us whether we stuck to the constraints around the relations for each of the jobs. So lets just build a data-frame of allowable compartments per job and join it to the normal response


In [ ]:
cols = ['jobId', 'allowableCompartments']
jdata = pandas.DataFrame(columns=cols)

idx = 0
for i, j in enumerate(m.jobs):
    jdata.loc[idx] = list([j.id, ','.join(j.compartmentRelations.compartmentIds)])
    idx +=1

nodes = t['nodes']

pandas.merge(nodes[['taskId', 'compartmentId', 'jobId']],jdata,left_on='jobId', right_on='jobId', how = 'left')


So that's nice, we can see that at each task assignment we only used compartments which were in the allowable set we provided the api. It's probably worth noting that the default is all compartments in a compartment-set are allowed so you can either specify an inclusive sub-set, or excluded sub-set.
If all compartments are excluded then the api will let you know that there's no feasible allocation.


### What next?
If you've made it to this point and worked through the ivr7 examples then you're probably ready to implement you own model!
Good luck and if you need any help feel free to pop us a mail (info@icepack.co) or open a github issue with your query.